<a href="https://colab.research.google.com/github/randywreed/sav2csv/blob/master/variable_sav_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
!pip install voila
!jupyter serverextension enable voila —sys-prefix
!jupyter nbextension enable --py widgetsnbextension
!pip install ipython_blocking

You should consider upgrading via the '/home/randy/anaconda3/bin/python -m pip install --upgrade pip' command.
Enabling: voila
- Writing config: /home/randy/.jupyter
    - Validating...
      voila 0.2.3 OK
Enabling: —sys-prefix
- Writing config: /home/randy/.jupyter
    - Validating...
Error loading server extension —sys-prefix
      X is —sys-prefix importable?
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


  Created wheel for ipython-blocking: filename=ipython_blocking-0.2.1-py3-none-any.whl size=6028 sha256=d5670d80ed0d85e45579f8acbd35aaeb4a4e7c56ccfb339e69e78e62e7a6cb53
  Stored in directory: /home/randy/.cache/pip/wheels/9f/03/12/e626994b2ba0aad81c32af1dc469dfb4442d4001392f34eab4
Successfully built ipython-blocking
You should consider upgrading via the '/home/randy/anaconda3/bin/python -m pip install --upgrade pip' command.


In [76]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/randy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [77]:
!pip install pyreadstat
import pandas as pd

You should consider upgrading via the '/home/randy/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import ipywidgets as widgets
import ipython_blocking

In [2]:
from IPython.display import display

In [18]:
upload=widgets.FileUpload(accept=".sav",multiple=False)
print('click continue button to continue after upload')
button=widgets.Button(description="continue")
display(upload,button)


click continue button to continue after upload


FileUpload(value={}, accept='.sav', description='Upload')

Button(description='continue', style=ButtonStyle())

<IPython.core.display.Javascript object>

In [21]:

%blockrun button

In [22]:

with open (upload.metadata[0]['name'],"w+b") as i:
  i.write(upload.data[0])

In [23]:
import pyreadstat
df,meta=pyreadstat.read_sav(upload.metadata[0]['name'])

In [24]:
df.colmnns=meta.column_names

/home/randy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [25]:
klab={}
klab=meta.column_names_to_labels
#type(klab)

In [26]:
labs=[]
for k,v in klab.items():
  #print("{}:{}".format(k,v))
  labs.append([k,v])

In [27]:
df_enriched=pyreadstat.set_value_labels(df,meta,formats_as_category=True)

In [28]:
def strip_it(initialsent):
  import string, re
  en_stops=set(stopwords.words('english'))
  #update stopwords
  en_stops.remove('not')
  newstopwords=['what','why','who','when','where','how','many']
  en_stops.update(newstopwords)
  #remove puntuation, stopwords, and add spaces to runon words 
  translator = str.maketrans('', '', string.punctuation)
  newvar=initialsent.translate(translator)
  #print('after punctuation strip=',newvar)
  newvar=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', newvar)
  #print('add spaces if necessary=',newvar)
  c_split=newvar.split(" ")
  c_strip=""
  c_stripns=""
  for word in c_split:
    if str.lower(word) not in en_stops:
      c_strip=c_strip+word+" "
      c_stripns=c_stripns+word+"_"
  c_stripns=c_stripns.rstrip('_')
  return (c_stripns,c_strip)


In [29]:
def create_options(lab,val):
  out=[]
  ret=strip_it(val)
  out=[lab,ret[0],ret[1],"other"]
  return out

In [30]:
import os
base=os.path.basename(upload.metadata[0]['name'])
newfile=os.path.splitext(base)
#print(newfile[0])
df_enriched.to_csv(newfile[0]+".csv")

In [31]:
from IPython.display import FileLink
# Import a module to delete the file
import os
# Create a download function
def csv_download_link(df, csv_file_name, delete_prompt=True):
    """Display a download link to load a data frame as csv within a Jupyter notebook

    Parameters
    ----------
    df : pandas data frame
    csv_file_name : str
    delete_prompt : bool
    """
    df.to_csv(csv_file_name, index=False)
    display(FileLink(csv_file_name))
    if delete_prompt:
        a = input('Right-click link to download, then Press enter to delete the file after you have downloaded it.')
        os.remove(csv_file_name)


In [32]:
if 'google.colab' in str(get_ipython()):
  from google.colab import files
  files.download(newfile[0]+'.csv')
else:
  csv_download_link(df_enriched,newfile[0]+'.csv')

/home/randy/sav2csv/Measuring Morality Study, 2012.csv

Right-click link to download, then Press enter to delete the file after you have downloaded it.
